# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 2

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task2`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [11]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F2JVHXWME to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "b4e518fe-3bdc-4371-a4be-7f3d83d7acbd",
    "id": "7fc54e95-14cf-4a6f-b50e-d6155c241309",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "d0ecd01b-d782-448e-bae0-c3cad0e0543a"
      },
      {
        "tenantId": "94c4857e-1130-4ab8-8eac-069b40c9db20"
      },
      {
        "tenantId": "f702a9dc-ae48-4dc7-8f0a-8155a6dfa4e5"
      }
    ],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "b4e518fe-3bdc-4371-a4be-7f3d83d7acbd",
    "user": {
      "name": "hachall@hotmail.com",
      "type": "user"
    }
  }
]


## Step 1. Write the code

In [25]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

In [26]:
teamname="schrodingers-duck"  # Update this field with your team name
task=["task2"]
slack_id="U04L3SFUCQ2"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [27]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task2_DumpMachineWrapper : qsharp.QSharpCallable = None
Task2_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 2 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [38]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Bitwise;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

// Task 2. Celebrate MIT iQuHack!
// (input will contain 5 qubits)

operation old(input : Qubit[], target : Qubit) : Unit is Adj {
    ControlledOnInt(13, X)(input, target);  // M
    ControlledOnInt( 9, X)(input, target);  // I
    ControlledOnInt(20, X)(input, target);  // T

    ControlledOnInt( 9, X)(input, target);  // I
    ControlledOnInt(17, X)(input, target);  // Q
    ControlledOnInt(21, X)(input, target);  // U
    ControlledOnInt( 8, X)(input, target);  // H
    ControlledOnInt( 1, X)(input, target);  // A
    ControlledOnInt( 3, X)(input, target);  // C
    ControlledOnInt(11, X)(input, target);  // K
}

operation Task2(input : Qubit[], target : Qubit) : Unit is Adj {
    ControlledOnInt(13, X)(input, target);  // M
    ControlledOnInt(20, X)(input, target);  // T
    ControlledOnInt( 8, X)(input, target);  // H
    ControlledOnInt( 1, X)(input, target);  // A
    ControlledOnInt(9, X)([input[0],input[1],input[3],input[4]], target);
    ControlledOnInt(3, X)([input[0],input[1],input[2],input[4]], target);
}

operation QubitArrayWrapperOperation (op : ((Qubit[], Qubit) => Unit is Adj), qs : Qubit[]) : Unit is Adj {        
    op(Most(qs), Tail(qs));
}

operation TestCompoundGate () : Unit {
    AssertOperationsEqualReferenced(5+1, QubitArrayWrapperOperation(Task2, _), QubitArrayWrapperOperation (old, _));
}


In [39]:
TestCompoundGate();

In [40]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task2_DumpMachineWrapper() : Unit {
    let N = 5;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task2(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task2_ResourceEstimationWrapper() : Unit {
    let N = 5;
    use (input, target) = (Qubit[N], Qubit());
    Task2(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [41]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
# qsharp.config["dump.truncateSmallAmplitudes"]=True
Task2_DumpMachineWrapper.simulate()

|000000⟩	0.17677669529663698 + 0𝑖
|000001⟩	0 + 0𝑖
|000010⟩	0.17677669529663698 + 0𝑖
|000011⟩	0 + 0𝑖
|000100⟩	0 + 0𝑖
|000101⟩	0.17677669529663698 + 0𝑖
|000110⟩	0.17677669529663698 + 0𝑖
|000111⟩	0 + 0𝑖
|001000⟩	0.17677669529663698 + 0𝑖
|001001⟩	0 + 0𝑖
|001010⟩	0 + 0𝑖
|001011⟩	0.17677669529663698 + 0𝑖
|001100⟩	0.17677669529663698 + 0𝑖
|001101⟩	0 + 0𝑖
|001110⟩	0.17677669529663698 + 0𝑖
|001111⟩	0 + 0𝑖
|010000⟩	0.17677669529663698 + 0𝑖
|010001⟩	0 + 0𝑖
|010010⟩	0.17677669529663698 + 0𝑖
|010011⟩	0 + 0𝑖
|010100⟩	0.17677669529663698 + 0𝑖
|010101⟩	0 + 0𝑖
|010110⟩	0.17677669529663698 + 0𝑖
|010111⟩	0 + 0𝑖
|011000⟩	0.17677669529663698 + 0𝑖
|011001⟩	0 + 0𝑖
|011010⟩	0.17677669529663698 + 0𝑖
|011011⟩	0 + 0𝑖
|011100⟩	0.17677669529663698 + 0𝑖
|011101⟩	0 + 0𝑖
|011110⟩	0.17677669529663698 + 0𝑖
|011111⟩	0 + 0𝑖
|100000⟩	0 + 0𝑖
|100001⟩	0.17677669529663698 + 0𝑖
|100010⟩	0 + 0𝑖
|100011⟩	0.17677669529663698 + 0𝑖
|100100⟩	0.17677669529663698 + 0𝑖
|100101⟩	0 + 0𝑖
|100110⟩	0.17677669529663698 + 0𝑖
|100111⟩	0 + 0𝑖
|101000⟩	0.17677669529663698 + 0𝑖
|101001⟩	0 + 0𝑖
|101010⟩	0 + 0𝑖
|101011⟩	0.17677669529663698 + 0𝑖
|101100⟩	0 + 0𝑖
|101101⟩	0.17677669529663698 + 0𝑖
|101110⟩	0.17677669529663698 + 0𝑖
|101111⟩	0 + 0𝑖
|110000⟩	0 + 0𝑖
|110001⟩	0.17677669529663698 + 0𝑖
|110010⟩	0.17677669529663698 + 0𝑖
|110011⟩	0 + 0𝑖
|110100⟩	0 + 0𝑖
|110101⟩	0.17677669529663698 + 0𝑖
|110110⟩	0.17677669529663698 + 0𝑖
|110111⟩	0 + 0𝑖
|111000⟩	0.17677669529663698 + 0𝑖
|111001⟩	0 + 0𝑖
|111010⟩	0.17677669529663698 + 0𝑖
|111011⟩	0 + 0𝑖
|111100⟩	0.17677669529663698 + 0𝑖
|111101⟩	0 + 0𝑖
|111110⟩	0.17677669529663698 + 0𝑖
|111111⟩	0 + 0𝑖

()

## Step 3. Evaluate the code using resource estimation

In [42]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="/subscriptions/7fc54e95-14cf-4a6f-b50e-d6155c241309/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/azure-q",
    location="UK South",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace azure-q in location uksouth.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 178071},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 409594},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 4},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 70},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 167},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current_availa

In [43]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [44]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task2_ResourceEstimationWrapper, jobName="RE for the task 2")

Submitting Task2_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 2
   Job ID: 4e3f0c4e-e8f0-49d5-99bf-d4d7d51bc4f7
Waiting up to 30 seconds for Azure Quantum job to complete...
[1:12:00 PM] Current job status: Executing
[1:12:06 PM] Current job status: Succeeded


In [45]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

{'errorBudget': {'logical': 0.0005, 'rotations': 0.0, 'tstates': 0.0005},
 'jobParams': {'errorBudget': 0.001,
  'qecScheme': {'crossingPrefactor': 0.03,
   'errorCorrectionThreshold': 0.01,
   'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
   'name': 'surface_code',
   'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance'},
  'qubitParams': {'instructionSet': 'GateBased',
   'name': 'qubit_gate_ns_e3',
   'oneQubitGateErrorRate': 0.001,
   'oneQubitGateTime': '50 ns',
   'oneQubitMeasurementErrorRate': 0.001,
   'oneQubitMeasurementTime': '100 ns',
   'tGateErrorRate': 0.001,
   'tGateTime': '50 ns',
   'twoQubitGateErrorRate': 0.001,
   'twoQubitGateTime': '50 ns'}},
 'logicalCounts': {'ccixCount': 30,
  'cczCount': 10,
  'measurementCount': 30,
  'numQubits': 9,
  'rotationCount': 0,
  'rotationDepth': 0,
  'tCount': 0},
 'logicalQubit': {'codeDistance': 11,
  'logicalCycleTime': 4400.0,
  'logicalErrorRate': 3.000000000000002e-08,
  'physicalQubits': 242},
 'physicalCounts': {'breakdown': {'algorithmicLogicalDepth': 150,
   'algorithmicLogicalQubits': 28,
   'cliffordErrorRate': 0.001,
   'logicalDepth': 150,
   'numTfactories': 12,
   'numTfactoryRuns': 14,
   'numTsPerRotation': None,
   'numTstates': 160,
   'physicalQubitsForAlgorithm': 6776,
   'physicalQubitsForTfactories': 77760,
   'requiredLogicalQubitErrorRate': 1.1904761904761904e-07,
   'requiredLogicalTstateErrorRate': 3.125e-06},
  'physicalQubits': 84536,
  'runtime': 660000},
 'physicalCountsFormatted': {'codeDistancePerRound': '9',
  'errorBudget': '1.00e-3',
  'errorBudgetLogical': '5.00e-4',
  'errorBudgetRotations': '0.00e0',
  'errorBudgetTstates': '5.00e-4',
  'logicalCycleTime': '4us 400ns',
  'logicalErrorRate': '3.00e-8',
  'numTsPerRotation': 'No rotations in algorithm',
  'numUnitsPerRound': '2',
  'physicalQubitsForTfactoriesPercentage': '91.98 %',
  'physicalQubitsPerRound': '6480',
  'requiredLogicalQubitErrorRate': '1.19e-7',
  'requiredLogicalTstateErrorRate': '3.13e-6',
  'runtime': '660us',
  'tfactoryRuntime': '46us 800ns',
  'tfactoryRuntimePerRound': '46us 800ns',
  'tstateLogicalErrorRate': '2.17e-6',
  'unitNamePerRound': '15-to-1 space efficient logical'},
 'reportData': {'assumptions': ['_More details on the following lists of assumptions can be found in the paper [Accessing requirements for scaling quantum computers and their applications](https://aka.ms/AQ/RE/Paper)._',
   '**Uniform independent physical noise.** We assume that the noise on physical qubits and physical qubit operations is the standard circuit noise model. In particular we assume error events at different space-time locations are independent and that error rates are uniform across the system in time and space.',
   '**Efficient classical computation.** We assume that classical overhead (compilation, control, feedback, readout, decoding, etc.) does not dominate the overall cost of implementing the full quantum algorithm.',
   '**Extraction circuits for planar quantum ISA.** We assume that stabilizer extraction circuits with similar depth and error correction performance to those for standard surface and Hastings-Haah code patches can be constructed to implement all operations of the planar quantum ISA (instruction set architecture).',
   '**Uniform independent logical noise.** We assume that the error rate of a logical operation is approximately equal to its space-time volume (the number of tiles multiplied by the number of logical time steps) multiplied by the error rate of a logical qubit in a standard one-tile patch in one logical time step.',
   '**Negligible Clifford costs for synthesis.** We assume that the space overhead for synthesis and space and time overhead for transport of magic states within magic state factories and to synthesis qubits are all negligible.',
   '**Smooth magic state consumption rate.** We assume that the rate of T state consumption throughout the compiled algorithm is almost cons

In [46]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [47]:
evaluate_results(result)

Logical algorithmic qubits = 28
Algorithmic depth = 150
Score = 4200


4200